In [1]:
import glob,os
import gzip

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import font_manager
plt.switch_backend('agg')
import seaborn as sns
sns.set_style( "white" )

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager
from matplotlib import font_manager
from matplotlib.colors import LinearSegmentedColormap

font_dirs = ['/home/jiangquanlong/miniconda3/envs/r_env/lib/python3.12/site-packages/matplotlib/mpl-data/fonts/ttf']
font_files = font_manager.findSystemFonts(fontpaths=font_dirs)
for font_file in font_files:
    font_manager.fontManager.addfont(font_file)

plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Arial']  
plt.rcParams['pdf.fonttype'] = 42

In [2]:
def get_ref_dict(data_name):
    if data_name != 'data9_a2i':
        f1 = '/disk1/0.Genome/gencode.v31/AS_events/SE.lenient.SEpos.txt'
        f2 = '/disk1/0.Genome/gencode.v31/AS_events/SE.lenient.5end_3end.txt'
    else:
        f1 = '/disk1/0.Genome/gencode.vM25/AS_events/SE.lenient.SEpos.txt'
        f2 = '/disk1/0.Genome/gencode.vM25/AS_events/SE.lenient.5end_3end.txt'
    SE_dict = {}
    for line in open(f1):
        line = line.strip().split('\t')
        SE_dict[line[0]] = line[1] + '_' + line[2] + '_' + line[3]
    
    end_dict = {}
    for line in open(f2):
        line = line.strip().split('\t')
        end_dict[line[0]] = line[1] + '_' + line[2] + '_' + line[3]
    return SE_dict,end_dict

def get_sc(name,fname):
    SE_dict,end_dict = get_ref_dict(name)
    df = pd.read_table(fname,header=0)
    fdr_col = df.columns[-1]
    df['SE_pos'] = df['GeneID'].map(SE_dict)
    df['5end_3end'] = df['GeneID'].map(end_dict)
    df = df.sort_values(by=fdr_col)
    return df[['SE_pos','5end_3end',fdr_col]]


In [3]:
def get_rmats(fname):
    df = pd.read_table(fname, header=0)
    df['3end'] = df['chr'].astype(str) + '_' + df['downstreamES'].astype(str)
    df['SE_pos'] = df['chr'].astype(str) + '_' + df['exonStart_0base'].astype(str) + '_' + df['exonEnd'].astype(str)
    df['5end_3end_pos'] = df['chr'].astype(str) + '_' + df['upstreamEE'].astype(str) + '_' + df['downstreamES'].astype(str)
    #df = df[abs(df['IncLevelDifference'])>0.1]
    df = df.sort_values(by='FDR')
    return df[['3end','SE_pos','5end_3end_pos','IncLevelDifference','FDR']]

In [4]:
def get_leaf(fname):
    pos_f = os.path.split(fname)[0] + '/leafcutter_cluster/sample_perind_numers.counts.gz'
    with gzip.open(pos_f, 'rt') as file:  
        lines = list(map(lambda l: l.strip().split(' '), file.readlines()))[1:]
    clu_dict = {}
    for line in lines:
        chrom,start,end,clu_id = line[0].split(':')
        clu_id = chrom + ':' + clu_id
        if clu_id not in clu_dict:
            clu_dict[clu_id] = [[],[]]
        clu_dict[clu_id][0].append('_'.join([chrom,start,end]))
        clu_dict[clu_id][1].append('_'.join([chrom,end]))
    
    df = pd.read_table(fname,header=0)
    pos_list = []
    three_end_list = []
    for clu_id in df['cluster']:
        poss = clu_dict[clu_id][0]; three_ends = clu_dict[clu_id][1]
        pos_list.append(poss)
        three_end_list.append(three_ends)
    df['5end_3end'] = pos_list
    df['3end'] = three_end_list
    df['5end_3end_tuple'] = df['5end_3end'].apply(tuple)
    df = df.sort_values('p.adjust').drop_duplicates(subset='5end_3end_tuple', keep='first')
    df =  df.sort_values(by='p.adjust')
    return df[['cluster','5end_3end','3end','p.adjust']]
        

In [5]:
def find_similar_intervals_rmats(list_A, list_B, tolerance=3):
    def parse_interval(interval):
        parts = interval.split('_')
        return parts[0], int(parts[1]), int(parts[2])
    
    # 预处理list_B，创建更高效的数据结构
    b_intervals_by_chrom = {}
    for interval_b in list_B:
        chrom_b, start_b, end_b = parse_interval(interval_b)
        if chrom_b not in b_intervals_by_chrom:
            b_intervals_by_chrom[chrom_b] = []
        b_intervals_by_chrom[chrom_b].append((start_b, end_b))
    
    common_intervals = []
    
    # 使用集合进行快速查找
    for interval_a in list_A:
        chrom_a, start_a, end_a = parse_interval(interval_a)
        
        # 快速检查染色体是否存在
        if chrom_a not in b_intervals_by_chrom:
            continue
        
        # 使用任何匹配即可的策略
        for start_b, end_b in b_intervals_by_chrom[chrom_a]:
            if (abs(start_a - start_b) <= tolerance) or (abs(end_a - end_b) <= tolerance):
                common_intervals.append(interval_a)
                break
    
    return common_intervals

In [6]:
def find_similar_intervals_leaf(list_A, list_B, tolerance=3):
    def parse_interval(interval):
        parts = interval.split('_')
        return parts[0], int(parts[1]), int(parts[2])
    
    # 预处理list_B，创建更高效的数据结构
    b_intervals_by_chrom = {}
    for interval_b in list_B:
        chrom_b, start_b, end_b = parse_interval(interval_b)
        if chrom_b not in b_intervals_by_chrom:
            b_intervals_by_chrom[chrom_b] = []
        b_intervals_by_chrom[chrom_b].append((start_b, end_b))
    
    common_intervals = []
    
    for interval_a_list in list_A:
        found = False
        for interval_a in interval_a_list:
            chrom_a, start_a, end_a = parse_interval(interval_a)
            if chrom_a not in b_intervals_by_chrom:
                continue
            for start_b, end_b in b_intervals_by_chrom[chrom_a]:
                if (abs(start_a - start_b) <= tolerance) or (abs(end_a - end_b) <= tolerance):
                    common_intervals.append(interval_a)
                    found = True
                    break
            if found:
                break
    
    return common_intervals

In [7]:
def remove_duplicates(lst):
    seen = set()
    return [x for x in lst if not (x in seen or seen.add(x))]

In [8]:
def compute_aucc(ranked_genes_bulk, ranked_genes_sc, k,tool, plot=False):
    ranked_genes_sc = remove_duplicates( ranked_genes_sc)
    intersection_counts = []
    for i in range(1, k + 1):
        top_bulk = ranked_genes_bulk[:i]
        top_sc = ranked_genes_sc[:i]
        if tool == 'rmats':
            intersection = len(find_similar_intervals_rmats(top_bulk, top_sc))
        else:
            intersection = len(find_similar_intervals_leaf(top_bulk, top_sc))
        intersection_counts.append(intersection)

    # 计算面积（Raw AUCC）
    raw_aucc = sum(intersection_counts)

    # 归一化：最大面积是 k*(k+1)/2
    max_area = k * (k + 1) / 2
    normalized_aucc = raw_aucc / max_area

    if plot:
        plt.figure(figsize=(6, 4))
        plt.plot(range(1, k + 1), intersection_counts, label='Concordance Curve', color='blue')
        plt.fill_between(range(1, k + 1), intersection_counts, alpha=0.3, color='blue')
        plt.title(f'Concordance Curve (AUCC = {normalized_aucc:.3f})')
        plt.xlabel('Top-k Genes')
        plt.ylabel('Intersection Size')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    return normalized_aucc

In [9]:
def compute_aucc_gene(name,ranked_genes_bulk, ranked_genes_sc, k,tool, plot=False):
    def find_same_gene(list_A,list_B):
        common_genes = []
        for gene in list_A:
            if gene in list_B:
                common_genes.append(gene)
        return common_genes
        
    ranked_genes_sc = remove_duplicates(ranked_genes_sc)
    #convert to gene id
    if name == 'data9_a2i':
        ranked_genes_sc = convert_to_geneid(ranked_genes_sc[:k+1],mm_anno_df)
        ranked_genes_bulk = convert_to_geneid(ranked_genes_bulk[:k+1],mm_anno_df)
    else:
        ranked_genes_sc = convert_to_geneid(ranked_genes_sc[:k+1],hg_anno_df)
        ranked_genes_bulk = convert_to_geneid(ranked_genes_bulk[:k+1],hg_anno_df)
    intersection_counts = []
    for i in range(1, k + 1):
        top_bulk = ranked_genes_bulk[:i]
        top_sc = ranked_genes_sc[:i]
        intersection = len(find_same_gene(top_bulk, top_sc))
        intersection_counts.append(intersection)

    # 计算面积（Raw AUCC）
    raw_aucc = sum(intersection_counts)

    # 归一化：最大面积是 k*(k+1)/2
    max_area = k * (k + 1) / 2
    normalized_aucc = raw_aucc / max_area

    if plot:
        plt.figure(figsize=(6, 4))
        plt.plot(range(1, k + 1), intersection_counts, label='Concordance Curve', color='blue')
        plt.fill_between(range(1, k + 1), intersection_counts, alpha=0.3, color='blue')
        plt.title(f'Concordance Curve (AUCC = {normalized_aucc:.3f})')
        plt.xlabel('Top-k Genes')
        plt.ylabel('Intersection Size')
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.tight_layout()
        plt.show()

    return normalized_aucc

In [10]:
def convert_to_geneid(positions,df):
    result_genes = []
    
    for pos in positions:
        if isinstance(pos,list):
            pos = pos[0]
        pos = pos.split('_')
        if len(pos) == 3:
            chrom, start, end = pos
        elif len(pos) == 2:
            chrom, start = pos
        else:
            pass
        
        start = int(start);
        # 找到相同染色体的基因
        chr_genes = df[df['chrom'] == chrom].copy()
        
        if len(chr_genes) == 0:
            result_genes.append("No gene found")
            continue
            
        # 找到包含start位置的基因
        containing_genes = chr_genes[
            (chr_genes['start'] <= start) & (chr_genes['end'] >= start)
        ].copy()
        
        if len(containing_genes) == 0:
            result_genes.append("No gene found")
            continue
            
        # 如果有protein_coding的基因，优先选择protein_coding
        protein_coding = containing_genes[containing_genes['gene_type'] == 'protein_coding']
        
        if len(protein_coding) > 0:
            # 如果有多个protein_coding基因，选择第一个
            best_gene = protein_coding.iloc[0]
        else:
            # 如果没有protein_coding，选择第一个基因
            best_gene = containing_genes.iloc[0]
        
        result_genes.append(best_gene['gene_id'])
    
    return result_genes

In [11]:
def get_anno_df(f):
    df = pd.read_csv(f, sep='\t', usecols=['gene_id', 'gene_type', 'position'])
    df[['chrom', 'coords']] = df['position'].str.split(':', expand=True)
    df[['start', 'end']] = df['coords'].str.split('-', expand=True).astype(int)
    return df

In [12]:
hg_anno_df = get_anno_df(f='/disk1/0.Genome/gencode.v31/GenomeAnnotation.txt')
mm_anno_df = get_anno_df(f='/disk1/0.Genome/gencode.vM25/GenomeAnnotation.txt')

In [13]:
sc_files = {'data2_24h':'/disk4/humanData/2.stemCell_GSE52529/SE_events/brie_quant_cell.Myoblast_Cell_T0_vs_Myoblast_Cell_T24.brie_ident.tsv',
           'data2_48h':'/disk4/humanData/2.stemCell_GSE52529/SE_events/brie_quant_cell.Myoblast_Cell_T0_vs_Myoblast_Cell_T48.brie_ident.tsv',
           'data2_72h':'/disk4/humanData/2.stemCell_GSE52529/SE_events/brie_quant_cell.Myoblast_Cell_T0_vs_Myoblast_Cell_T72.brie_ident.tsv',
           'data8_ipsc':'/disk4/humanData/3.iPSC_GSE85908/SE_events/brie_quant_cell.Motor_neurons_vs_Induced_pluripotent_cells.brie_ident.tsv',
           'data9_a2i':'/disk4/mouseData/CSC_E-MTAB-2600/SE_events/brie_quant_cell.serum_vs_a2i.brie_ident.tsv'}

In [14]:
rmats_files = {'data2_24h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_24hrs/SE.MATS.JCEC.txt',
               'data2_48h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_48hrs/SE.MATS.JCEC.txt',
               'data2_72h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_72hrs/SE.MATS.JCEC.txt',
              'data8_ipsc':'/disk4/humanData/3.iPSC_GSE85908/bulk_spl/Induced_pluripotent_cells/SE.MATS.JCEC.txt',
              'data9_a2i':'/disk4/mouseData/CSC_E-MTAB-2600/bulk_spl/serum_bulk/SE.MATS.JCEC.txt'}

In [15]:
leaf_files = {'data2_24h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_24hrs_groups_cluster_significance.txt',
             'data2_48h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_48hrs_groups_cluster_significance.txt',
             'data2_72h':'/disk4/humanData/2.stemCell_GSE52529/bulk_spl/bulk_72hrs_groups_cluster_significance.txt',
             'data8_ipsc':'/disk4/humanData/3.iPSC_GSE85908/bulk_spl/Induced_pluripotent_cells_groups_cluster_significance.txt',
             'data9_a2i':'/disk4/mouseData/CSC_E-MTAB-2600/bulk_spl/a2i_bulk_groups_cluster_significance.txt'}

In [21]:
result = {}
gene_result = {}
k = 100
for name in sc_files:
    print(name)
    result[name] = {}; gene_result[name] = {}
    sc_f = sc_files[name]; rmats_f = rmats_files[name]; leaf_f = leaf_files[name]
    sc_pval = get_sc(name,sc_f)
    rmats_pval = get_rmats(rmats_f)
    leaf_pval = get_leaf(leaf_f)
    tool = 'rmats'
    aucc_score = compute_aucc(rmats_pval['SE_pos'].to_list(),sc_pval['SE_pos'].to_list(), k=k,tool=tool)
    result[name][tool] = aucc_score
    tool = 'leaf'
    aucc_score = compute_aucc(leaf_pval['5end_3end'].to_list(),sc_pval['5end_3end'].to_list(), k=k,tool=tool)
    result[name][tool] = aucc_score
    #################gene level##################
    tool = 'rmats'
    aucc_score = compute_aucc_gene(name,rmats_pval['SE_pos'].to_list(),sc_pval['SE_pos'].to_list(), k=k,tool=tool)
    gene_result[name][tool] = aucc_score
    tool = 'leaf'
    aucc_score = compute_aucc_gene(name,leaf_pval['5end_3end'].to_list(),sc_pval['5end_3end'].to_list(), k=k,tool=tool)
    gene_result[name][tool] = aucc_score

data2_24h
data2_48h
data2_72h
data8_ipsc
data9_a2i


In [26]:
gene_result

{'data2_24h': {'rmats': 0.16772277227722773, 'leaf': 0.1093069306930693},
 'data2_48h': {'rmats': 0.17683168316831682, 'leaf': 0.14594059405940593},
 'data2_72h': {'rmats': 0.2732673267326733, 'leaf': 0.18831683168316832},
 'data8_ipsc': {'rmats': 0.14554455445544554, 'leaf': 0.11227722772277228},
 'data9_a2i': {'rmats': 0.21722772277227723, 'leaf': 0.09722772277227723}}

In [26]:
gene_result

{'data2_24h': {'rmats': 0.1339241516966068, 'leaf': 0.1078003992015968},
 'data2_48h': {'rmats': 0.17945708582834333, 'leaf': 0.1798003992015968},
 'data2_72h': {'rmats': 0.22409580838323354, 'leaf': 0.20709780439121755},
 'data8_ipsc': {'rmats': 0.21261477045908184, 'leaf': 0.14189221556886228},
 'data9_a2i': {'rmats': 0.19761277445109782, 'leaf': 0.14083033932135727}}

In [27]:
result

{'data2_24h': {'rmats': 0.06219560878243513, 'leaf': 0.04046307385229541},
 'data2_48h': {'rmats': 0.0857804391217565, 'leaf': 0.05337325349301397},
 'data2_72h': {'rmats': 0.10209980039920159, 'leaf': 0.06926147704590818},
 'data8_ipsc': {'rmats': 0.13305389221556888, 'leaf': 0.06089421157684631},
 'data9_a2i': {'rmats': 0.13069860279441117, 'leaf': 0.05996007984031936}}

In [23]:
df = pd.DataFrame.from_dict(result,orient='index')
df.to_csv('BRIE2_AUCC.{}.csv'.format(k))

In [24]:
df = pd.DataFrame.from_dict(gene_result,orient='index')
df.to_csv('BRIE2_AUCC.gene.{}.csv'.format(k))